In [1]:
import numpy as np
import pandas as pd
import math
import csv
import os
import datetime as dt
import yfinance
from sklearn.linear_model import LinearRegression as lreg
from matplotlib import pyplot as plt

In [2]:
stock_path = """C:\\Users\\eriks\\Documents\\Python_Practice\\Projects\\Stocks\\Updated\\stocks"""
etf_path = """C:\\Users\\eriks\\Documents\\Python_Practice\\Projects\\Stocks\\Updated\\etfs"""

In [4]:
def openstock(stock_tickers, etf_ticker, event_date, est_window_size, event_lead, event_lag):
    pd.options.mode.chained_assignment = 'raise'
    # Parse window
    event_date = dt.datetime.strptime(event_date, '%d-%m-%Y').date()
    est_start = event_date - dt.timedelta(days=est_window_size + event_lead + 1)
    event_start = event_date - dt.timedelta(days=event_lead)
    event_end = event_date + dt.timedelta(days=event_lag)
    # Stock
    all_windows = []
    for c, stock_ticker in enumerate(stock_tickers):
        stock_file = stock_ticker + '.csv'
        with open(stock_path + "\\" + stock_file, 'r') as f:
            c = stock_file.split('.',1)[0]
            read = csv.reader(f)
            lst = list(read)
            lst[0][4] = c
            stock_full = pd.DataFrame(lst[1:],columns=lst[0])
        stock_close = stock_full.iloc[:,[0,4]].copy()
        stock_close.loc[:,'Date'] = pd.to_datetime(stock_close.loc[:,'Date']).dt.date
        stock_close.loc[:,stock_ticker] = pd.to_numeric(stock_close.loc[:,stock_ticker]).round(2)
        stock_close.set_index(keys='Date', inplace=True)
        stock_window = stock_close[(stock_close.index >= est_start) & (stock_close.index <= event_end)].copy()
        stock_window.name = str(est_start) + ' - ' + str(event_end)
        stock_window = stock_window.pct_change().round(5)
        stock_window.loc[:,'Period'] = (stock_window.index - event_date).days
        stock_window.loc[:,'Period'] = 't' + stock_window.loc[:,'Period'].astype(str)
        stock_window = stock_window.set_index('Period').T
        all_windows.append(stock_window)
        
    # ETF
    if etf_ticker != '':
        etf_file = etf_ticker + '.csv'
        with open(etf_path + "\\" + etf_file, 'r') as f:
            c = etf_file.split('.',1)[0]
            read = csv.reader(f)
            lst = list(read)
            lst[0][4] = etf_ticker
            etf_full = pd.DataFrame(lst[1:],columns=lst[0])
            etf_close = etf_full.iloc[:,[0,4]].copy()
            etf_close.loc[:,'Date'] = pd.to_datetime(etf_close.loc[:,'Date']).dt.date
            etf_close.loc[:,etf_ticker] = pd.to_numeric(etf_close.loc[:,etf_ticker]).round(2)
            etf_close.set_index(keys='Date', inplace=True)
            etf_window = etf_close[(etf_close.index >= est_start) & (etf_close.index <= event_end)].copy()
            etf_window.name = str(est_start) + ' - ' + str(event_end)
            etf_window = etf_window.pct_change().round(5)
            etf_window.loc[:,'Period'] = (etf_window.index - event_date).days
            etf_window.loc[:,'Period'] = 't' + etf_window.loc[:,'Period'].astype(str)
            etf_window = etf_window.set_index('Period').T
    #     return etf_window
        all_windows.insert(0, etf_window)
    else:
        all_windows = pd.concat(all_windows)
        all_windows.drop(all_windows.columns[0], axis=1, inplace=True)
    return all_windows

In [5]:
with open(os.getcwd() + '\\NASDAQ_Banks.csv', 'r') as f:
    read = csv.reader(f)
    lst = list(read)
    all_banks = pd.DataFrame(lst[1:],columns=lst[0])
    all_banks.columns = ['Rank', 'Bank', 'Market cap', 'Ticker']

In [6]:
tickers = list(all_banks['Ticker'])

In [7]:
tickers = [e for e in tickers if e not in ('BBT', 'STI')]

In [8]:
returns = openstock(tickers, '', '05-03-2020', 100, 5, 5)   #09-11-2016  #['JPM', 'C', 'WFC', 'BAC']

In [9]:
returns

Period,t-105,t-104,t-101,t-100,t-99,t-97,t-94,t-93,t-92,t-91,...,t-8,t-7,t-6,t-3,t-2,t-1,t0,t1,t4,t5
JPM,0.00231,0.00662,0.00535,0.00137,0.00296,-0.00227,-0.00220,-0.01285,0.01973,0.00544,...,0.00301,-0.04161,-0.04334,0.04659,-0.03752,0.02471,-0.04906,-0.05168,-0.13546,0.07770
BAC,0.00459,0.01035,0.00874,-0.00359,0.00210,-0.00299,0.00330,-0.01765,0.00944,-0.00181,...,-0.01448,-0.04897,-0.02163,0.03053,-0.05516,0.02306,-0.05671,-0.03996,-0.14702,0.07661
WFC,0.00037,0.01344,-0.00129,-0.00738,0.00985,0.00221,-0.01542,-0.01940,0.01255,-0.00019,...,-0.02172,-0.03783,-0.03814,0.03452,-0.04094,0.02147,-0.06039,-0.04653,-0.12429,0.08005
C,-0.00014,0.01313,0.01082,-0.00291,0.00345,-0.00792,-0.00799,-0.01597,0.01227,0.00647,...,-0.01800,-0.05471,-0.01536,0.06508,-0.03758,0.03597,-0.05787,-0.03481,-0.16172,0.08195
MS,-0.00530,0.00839,0.01442,-0.01101,0.00749,-0.00603,-0.00808,-0.01976,0.01517,0.00471,...,-0.01645,-0.03874,-0.00837,0.03220,-0.04475,0.01892,-0.05858,-0.01761,-0.10373,0.05333
GS,0.00147,0.00939,0.01121,-0.00135,0.00225,-0.00718,-0.01676,-0.02481,0.01743,0.00556,...,-0.00841,-0.04676,-0.02392,0.04333,-0.02883,0.02610,-0.04767,-0.02988,-0.10391,0.06678
USB,0.00643,0.00941,-0.00383,0.00167,0.00451,-0.00266,-0.00450,-0.01389,0.00645,0.00489,...,-0.02090,-0.04309,-0.02273,0.03704,-0.04277,0.01215,-0.06172,-0.03038,-0.14441,0.06388
AXP,-0.00419,0.00219,0.01293,-0.00672,0.00451,-0.00175,-0.02381,-0.00588,0.01175,0.00288,...,-0.01985,-0.04802,-0.02553,0.03584,-0.05146,0.07120,-0.04114,-0.02434,-0.09193,0.05097
SCHW,0.07330,0.00354,0.02303,-0.00142,0.01219,-0.00682,0.00141,-0.01836,0.00760,-0.00591,...,-0.00433,0.01183,-0.02768,0.03018,-0.08766,-0.02977,-0.06351,-0.01925,-0.11310,0.08821
PNC,-0.00059,0.00765,0.00360,-0.00332,0.00262,-0.00052,-0.00803,-0.01474,0.01075,0.00562,...,-0.01861,-0.05832,-0.03231,0.04937,-0.05391,0.03116,-0.06731,-0.05353,-0.13552,0.09408


In [10]:
Er = list(returns.mean(axis=1)) #should use just the estimation window, currently uses all

In [11]:
R0 = list(returns['t0'])

In [12]:
events = pd.DataFrame({'Er':Er, 'R0':R0})

In [13]:
events['CARi'] = events['R0'] - events['Er']

In [14]:
avg = [events['Er'].mean(axis=0).round(5), 
       events['R0'].mean(axis=0).round(5), 
       events['CARi'].mean(axis=0).round(5)]

In [15]:
std = [events['Er'].std(axis=0).round(5), 
       events['R0'].std(axis=0).round(5), 
       events['CARi'].std(axis=0).round(5)]

In [16]:
sqrt = avg[2]/(std[2]/math.sqrt(len(events)))

In [17]:
stats  = pd.DataFrame({'Average':avg, 'StdDev':std, 't-stat':sqrt})

In [18]:
stats.index = ['Er','R0','CARi']

In [19]:
stats.T

,Er,R0,CARi
Average,-0.00385,-0.05352,-0.04967
StdDev,0.00140,0.01728,0.01676
t-stat,-18.50770,-18.50770,-18.50770


In [20]:
events

,Er,R0,CARi
0,-0.003085,-0.04906,-0.045975
1,-0.004027,-0.05671,-0.052683
2,-0.005399,-0.06039,-0.054991
3,-0.003417,-0.05787,-0.054453
4,-0.002651,-0.05858,-0.055929
5,-0.002013,-0.04767,-0.045657
6,-0.005434,-0.06172,-0.056286
7,-0.001679,-0.04114,-0.039461
8,-0.003713,-0.06351,-0.059797
9,-0.004205,-0.06731,-0.063105


To-Do:
* Describe T-Stat using jupyter math formulations:
$$e^{i\pi} + 1 = 0$$
$$e^x=\sum_{i=0}^\infty \frac{1}{i!}x^i$$
* Apply market model using regression on estimation window.